# Manifold Muon: Theory and Mathematical Foundations

**EECS 182 Final Project - Category 1: Optimizers & Hyperparameter Transfer**

This notebook provides the theoretical background for our project on spectral-norm constrained optimization and manifold Muon.

---

## Table of Contents
1. [Motivation: Why Control Spectral Norms?](#1-motivation)
2. [The Muon Inner Problem](#2-muon-inner)
3. [Inner Solver Algorithms](#3-inner-solvers)
4. [Stability Metrics and Diagnostics](#4-stability-metrics)
5. [muP and Hyperparameter Transfer](#5-mup)
6. [Project Codebase Overview](#6-codebase)

---
## 1. Motivation: Why Control Spectral Norms? <a name="1-motivation"></a>

### The Problem with Unconstrained Weight Dynamics

In deep neural network training, three types of tensors circulate: **activations**, **gradients**, and **weights**. During optimization, we want to keep their norms stable to avoid:

- **Exploding gradients/activations**: $\|\mathbf{h}^{(l)}\| \to \infty$ as depth increases
- **Vanishing gradients/activations**: $\|\mathbf{h}^{(l)}\| \to 0$ 

For a linear layer $\mathbf{h}^{(l+1)} = W^{(l)} \mathbf{h}^{(l)}$, the **spectral norm** $\|W\|_2 = \sigma_{\max}(W)$ directly controls signal amplification:

$$\|W \mathbf{x}\|_2 \leq \|W\|_2 \cdot \|\mathbf{x}\|_2$$

If $\|W^{(l)}\|_2 > 1$ for many layers, activations explode. If $\|W^{(l)}\|_2 < 1$, they vanish.

### Prior Approaches

| Method | Mechanism | Limitation |
|--------|-----------|------------|
| **Weight clipping** | Clip $W_{ij} \in [-c, c]$ | Hyperparameter $c$ is hard to tune |
| **Spectral normalization** | Rescale $W \leftarrow W / \sigma_{\max}(W)$ | Doesn't prevent rank collapse |
| **Orthogonal regularization** | Soft penalty $\|W^\top W - I\|_F^2$ | Doesn't guarantee constraint satisfaction |

### The Manifold Muon Approach

**Muon** (MomentUm Orthogonalized by Newton-Schulz) takes a different approach:

1. **Constrain the update step**, not just the weights
2. **Spectral norm of $\Delta W$** bounds how much $\|Wx\|$ can change for any unit $x$
3. Optionally: constrain $W$ to a **manifold** (e.g., Stiefel: $W^\top W = I$)

The key insight: **the step size $\eta$ directly bounds worst-case output change**:

$$\|(W + \Delta W)x - Wx\| = \|\Delta W \cdot x\| \leq \|\Delta W\|_2 \leq \eta$$

---
## 2. The Muon Inner Problem <a name="2-muon-inner"></a>

### Problem Formulation

At each optimization step, given gradient $G = \nabla_W \mathcal{L}$, we want to find the best update $A$ that:
1. Minimizes the linearized loss decrease: $\langle G, A \rangle$
2. Satisfies the spectral budget: $\|A\|_2 \leq \eta$
3. (Optionally) Stays tangent to the manifold: $A^\top W + W^\top A = 0$

$$\boxed{\min_{A \in \mathbb{R}^{m \times n}} \text{trace}(G^\top A) \quad \text{s.t.} \quad \|A\|_{\text{spectral}} \leq \eta \quad \text{and} \quad A^\top W + W^\top A = 0}$$

### Unconstrained Case (No Tangency)

Without the tangency constraint, the problem simplifies:

$$\min_{\|A\|_2 \leq \eta} \langle G, A \rangle$$

**Closed-form solution**: Let $G = U \Sigma V^\top$ be the SVD. The optimal $A^*$ is:

$$A^* = -\eta \cdot u_1 v_1^\top$$

where $u_1, v_1$ are the top singular vectors of $G$. This is a **rank-1 update** with spectral norm exactly $\eta$.

### Lagrangian Dual Formulation

For the constrained case, we form the Lagrangian:

$$\mathcal{L}(A, \lambda, \Lambda) = \langle G, A \rangle + \lambda(\|A\|_2 - \eta) + \langle \Lambda, A^\top W + W^\top A \rangle$$

The dual problem is:

$$\max_{\lambda \geq 0, \Lambda} \min_A \mathcal{L}(A, \lambda, \Lambda)$$

This is what **dual ascent** and **quasi-Newton** methods solve.

---
## 3. Inner Solver Algorithms <a name="3-inner-solvers"></a>

We implement several inner solvers, each with different trade-offs:

### 3.1 Spectral Clipping (Baseline)

**Simplest approach**: If $\|\Delta\|_2 > \eta$, rescale:

$$\Delta_{\text{clipped}} = \Delta \cdot \frac{\eta}{\|\Delta\|_2}$$

**Pros**: Fast, simple  
**Cons**: Doesn't respect tangent space, may distort gradient direction

### 3.2 Dual Ascent

Iterate on the dual variable $\lambda$:

1. For fixed $\lambda$: compute $A^*(\lambda)$ in closed form
2. Gradient ascent on $\lambda$: $\lambda \leftarrow \lambda + \alpha (\|A^*\|_2 - \eta)$

**Convergence**: Sublinear $O(1/t)$ for smooth duals

### 3.3 Quasi-Newton on the Dual

Replace gradient ascent with **L-BFGS** for faster convergence:

$$\lambda_{k+1} = \lambda_k + H_k^{-1} \nabla_\lambda d(\lambda_k)$$

where $H_k$ is approximated from recent $(s, y)$ pairs. Uses **warm-starting** across optimization steps.

**Convergence**: Superlinear for well-conditioned problems

### 3.4 Frank-Wolfe (Conditional Gradient)

**Projection-free** method: never explicitly project onto the constraint set.

At iteration $t$:
1. **Linear Minimization Oracle (LMO)**: $s_t = \arg\min_{\|A\|_2 \leq \eta} \langle \nabla f(A_t), A \rangle$
   - Solution: $s_t = -\eta \cdot u_1 v_1^\top$ (rank-1 atom from top SVD)
2. **Update**: $A_{t+1} = (1 - \gamma_t) A_t + \gamma_t s_t$

**Key property**: $A_t$ is a **convex combination of rank-1 atoms**, so it's **low-rank**!

**Convergence**: $O(1/t)$ sublinear, but each step is cheap (just top-SVD)

### 3.5 ADMM (Alternating Direction Method of Multipliers)

Split the problem into easier subproblems:

$$\min_A \langle G, A \rangle \quad \text{s.t.} \quad A = Z, \quad \|Z\|_2 \leq \eta$$

Augmented Lagrangian:
$$L_\rho(A, Z, U) = \langle G, A \rangle + \frac{\rho}{2}\|A - Z + U\|_F^2 + \mathbb{I}_{\|Z\|_2 \leq \eta}$$

ADMM iterations:
1. **A-update**: $A^{k+1} = Z^k - U^k - G/\rho$ (closed-form)
2. **Z-update**: $Z^{k+1} = \Pi_{\|\cdot\|_2 \leq \eta}(A^{k+1} + U^k)$ (spectral projection)
3. **U-update**: $U^{k+1} = U^k + A^{k+1} - Z^{k+1}$

**Convergence**: $O(1/t)$ with adaptive $\rho$

---
## 4. Stability Metrics and Diagnostics <a name="4-stability-metrics"></a>

### 4.1 Spectral Norm Tracking

We track $\sigma_{\max}(W^{(l)})$ for each layer to monitor:
- **Lipschitz constant** of each layer
- **Conditioning**: $\kappa(W) = \sigma_{\max} / \sigma_{\min}$
- **Rank collapse**: if small singular values vanish

### 4.2 Sharpness (SAM Proxy)

Sharpness measures how much the loss increases under small perturbations:

$$\text{Sharpness} \approx \mathcal{L}(w + \epsilon \cdot g/\|g\|) - \mathcal{L}(w)$$

**Connection to Hessian**: For small $\epsilon$, sharpness $\approx \frac{\epsilon^2}{2} \lambda_{\max}(H)$

Lower sharpness → flatter minima → better generalization (SAM hypothesis)

### 4.3 Gradient Noise Scale (GNS)

The **critical batch size** (CBS) determines when increasing batch size stops helping:

$$\text{GNS} = \frac{\text{tr}(\Sigma)}{\|\mu\|^2}$$

where $\mu$ is the true gradient and $\Sigma$ is the gradient covariance.

**Our proxy**: Using two minibatches $g_1, g_2$:

$$\widehat{\text{GNS}} = \frac{\|g_1 - g_2\|^2}{2}$$

### 4.4 Top Hessian Eigenvalue

We estimate $\lambda_{\max}(H)$ via **power iteration** with Hessian-vector products:

1. Initialize random $v$
2. Repeat: $v \leftarrow Hv / \|Hv\|$
3. Estimate: $\lambda_{\max} \approx v^\top H v$

**HVP computation** (Pearlmutter trick): $Hv = \nabla_w (\nabla_w \mathcal{L} \cdot v)$

### 4.5 Subspace Drift

Track how the **principal subspaces** of weight matrices change over training:

$$\text{Drift}(W_{\text{old}}, W_{\text{new}}) = \max_i \theta_i$$

where $\theta_i$ are the **principal angles** between the top-$k$ singular subspaces.

---
## 5. muP and Hyperparameter Transfer <a name="5-mup"></a>

### The Width Transfer Problem

Typically, optimal hyperparameters (learning rate, weight decay, etc.) change when we scale model width:

$$\text{Optimal LR for width } w \neq \text{Optimal LR for width } 2w$$

This makes hyperparameter tuning expensive for large models.

### muP (Maximal Update Parameterization)

**muP** (Yang & Hu, 2021) derives scaling rules so that:

$$\text{Optimal LR for width } w = \text{Optimal LR for any width}$$

Key insight: scale initialization, learning rates, and activations appropriately with width.

### Spectral Constraints and Transfer

**Our hypothesis**: Spectral-norm constraints provide an alternative route to hyperparameter transfer:

1. The spectral budget $\eta$ has a **width-independent interpretation**: max output change per step
2. If we set $\eta$ based on desired output perturbation, it should transfer across widths
3. This complements muP's initialization-based approach

### Experiment Design

We test this by:
1. Training MLPs at widths $w \in \{0.5, 0.75, 1.0, 1.5, 2.0\} \times$ base
2. Fixing all hyperparameters (LR, spectral budget)
3. Comparing final accuracy across widths

**If spectral constraints help**: accuracy should be stable across widths  
**If not**: accuracy will degrade as width increases

---
## 6. Project Codebase Overview <a name="6-codebase"></a>

### Directory Structure

```
muon_mve_project/
├── muon/
│   ├── __init__.py           # Module exports
│   ├── inner_solvers.py      # SpectralClip, FrankWolfe, DualAscent, QuasiNewton, ADMM
│   ├── muon_sgd.py           # MuonSGD, MuonAdamW optimizers
│   └── metrics.py            # Spectral norms, sharpness, GNS, Hessian eigenvalues
├── models/
│   └── __init__.py           # SmallCNN, ResNet18, TinyViT, MLPMixer, WidthScalableMLP
├── scripts/
│   ├── run_experiments.py    # Batch experiment runners
│   └── aggregate_results.py  # Multi-seed aggregation
├── notebooks/
│   ├── 01_theory_and_motivation.ipynb  # This notebook
│   ├── 02_experiments.ipynb            # Running experiments
│   └── 03_analysis.ipynb               # Results analysis
├── train.py                  # Main training script
├── plot_logs.py              # Plotting utilities
└── requirements.txt
```

### Key Components

| Component | Description |
|-----------|-------------|
| `MuonSGD` | SGD with pluggable inner solver for spectral constraints |
| `MuonAdamW` | AdamW variant with spectral constraints |
| `SpectralClipSolver` | Simple rescaling baseline |
| `DualAscentSolver` | Lagrangian dual ascent with warm-starting |
| `QuasiNewtonDualSolver` | L-BFGS on the dual |
| `FrankWolfeSolver` | Projection-free, low-rank atoms |
| `ADMMSolver` | Splitting method with adaptive ρ |

### Usage Pattern

```python
from muon import MuonSGD, get_inner_solver
from models import get_model

# Create model
model = get_model('resnet18', width_mult=1.0)

# Create optimizer with spectral constraints
solver = get_inner_solver('dual_ascent', max_iters=20)
optimizer = MuonSGD(
    model.parameters(),
    lr=0.1,
    momentum=0.9,
    spectral_budget=0.1,
    inner_solver=solver
)
```

In [ ]:
# Verify imports work
import sys
sys.path.insert(0, '..')

from muon import (
    MuonSGD, MuonAdamW,
    SpectralClipSolver, FrankWolfeSolver, DualAscentSolver,
    QuasiNewtonDualSolver, ADMMSolver,
    compute_spectral_norms, estimate_sharpness
)
from models import get_model, MODEL_REGISTRY

print("Available models:", list(MODEL_REGISTRY.keys()))
print("\nImports successful!")

---

## Next Steps

Continue to **02_experiments.ipynb** to run the actual experiments, or **03_analysis.ipynb** to analyze existing results.